In [ ]:
import re
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
file = open('chatExported.txt','r',encoding='utf-8')

In [ ]:
data = file.read()
# print(data)

In [ ]:
pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[ap]m\s-\s'

In [ ]:
messages = re.split(pattern,data)[1:  ]



In [ ]:
dates = re.findall(pattern,data)
dates

In [ ]:
dates = [date.strip(' -') for date in dates]
# messages = [date.strip('\n') for message in messages]
df = pd.DataFrame({'messages':messages,"dates":dates})
df.head()

In [ ]:

date_format = '%d/%m/%y, %I:%M %p'
df['dates'] = pd.to_datetime(df['dates'], format=date_format)
df.head()



In [ ]:
users = []
msgs = []

for message in df['messages']:
    splitting = re.split(':\s', message, maxsplit=1)
    
    if len(splitting) == 2:
        username, message_text = splitting
        users.append(username)
        msgs.append(message_text)
    else:
        users.append('message')
        msgs.append(message)








In [ ]:
df

In [ ]:
df['users'] = users
df2 = df
df['messages'] = msgs
df

In [ ]:

df['messages'] = msgs
df.head()

In [ ]:
df2['year'] = df2['dates'].dt.year
df2

In [ ]:
df2['month'] = df2['dates'].dt.month_name()
df2

In [ ]:
df2.drop(['year'],axis=1)


In [ ]:
df2['month'] = df2['dates'].dt.month_name()
df2

In [ ]:
df2['day'] = df2['dates'].dt.day
df2

In [ ]:
df2['hour'] = df2['dates'].dt.hour
df2

In [ ]:
df2['min'] = df2['dates'].dt.minute
df2.head()

In [ ]:
lst = df2['users'].unique()
lst

In [ ]:
df2[df2['users'] == 'Saatvii'].shape[0]

In [ ]:
messages = []
for message in df2['messages']:
#     print(message)
    messages.extend(message.split())
len(messages)

In [ ]:
media_count = df2[df2['messages'] == '<Media omitted>\n'].shape[0]
# df2[df2['messages'] == '<Media omitted>\n']
media_count

In [ ]:
media_ind = df2[df2['users'] == '👻']
media_ind[media_ind['messages'] ==  '<Media omitted>\n']



In [ ]:
pip install urlextract

In [ ]:
from urlextract import URLExtract

extractor = URLExtract()
urls = extractor.find_urls("Text with URLs. Let's have URL gmail.com as an example.")
print(urls) # prints: ['janlipovsky.cz']

In [ ]:
##Busiest person in the chat
df2
import matplotlib.pyplot as plt

In [ ]:
x = df2['users'].value_counts().head()

names = x.index
count = x.values
x

In [ ]:
busy = sns.barplot(data = df2,x=names,y=count)
plt.xticks(rotation="vertical")
busy.set_title("Top 5 Busiest Persons in the group")
busy.set_xlabel("User Names")
busy.set_ylabel("Messages Count")
plt.show()

In [ ]:

df3 = round((df2['users'].value_counts() / df2.shape[0]) * 100, 2).reset_index().rename(columns={'index': 'name', 'users': 'percent'})
df3

In [ ]:
df3.plot(kind="pie",labels ='name',values='percent')

In [ ]:
palette_color = sns.color_palette('bright')
  
# plotting data on chart
# plt.pie(data, labels=keys, colors=palette_color, autopct='%.0f%%')
pie = plt.pie(df3.head()['percent'],labels = df3.head()['name'],colors=palette_color,autopct='%0f%%',)


In [ ]:
pip install wordcloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
wc = WordCloud(width=500,height=500,min_font_size = 10,background_color='white')
df_wc = wc.generate(df2['messages'].str.cat(sep = " "))
df_wc

In [ ]:
img = plt.imshow(df_wc)

In [ ]:
temp = df2[df2['users'] != 'message']
temp = temp[temp['messages'] != "<Media omitted>\n"]
temp

In [ ]:
file = open('stopwords.txt','r')
stop_words = file.read()
stop_words


In [ ]:
words = []
for message in temp['messages']:
    for word in message.lower().split():
        if word not in stop_words:
            words.append(word)
words
    

In [ ]:
from collections import Counter
data = pd.DataFrame(Counter(words).most_common(20))
data.head()

In [ ]:
sns.barplot(data = data,x=data[1],y=data[0],orient="h")
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
pip install emoji

In [ ]:
import emoji
import re
temp['messages'].head()

In [ ]:
emojis = []
for message in temp['messages']:
    emojis_in_message = re.findall(r':(?:[^:\s]+):', emoji.demojize(message))
    emojis.extend(emojis_in_message)
emojis
actual_emojis = [emoji.emojize(emoji_text) for emoji_text in emojis]


In [ ]:
emoji_df = pd.DataFrame(Counter(actual_emojis).most_common(len(Counter(actual_emojis))))

In [ ]:
emoji_df.head()

In [ ]:
palette_color = sns.color_palette('dark')
plt.pie(emoji_df.head()[1],labels = emoji_df.head()[0],autopct = "%0.2f")
plt.show()

In [ ]:
sns.barplot(emoji_df[0].head(),emoji_df[1].head())
plt.show()

In [ ]:
df

In [ ]:
temp['month_num'] = temp['dates'].dt.month

In [ ]:
df = temp

In [ ]:
timeline =  df.groupby(['year','month_num','month']).count()['messages'].reset_index()
timeline

In [ ]:
time = []
for i in range(timeline.shape[0]):
    time.append(timeline['month'][i] + "-"+str(timeline['year'][i]))

In [ ]:
time

In [ ]:
timeline['time'] = time

In [ ]:
timeline

In [ ]:
sns.lineplot(x=timeline['time'],y=timeline['messages'])
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
df['only_date'] = df['dates'].dt.date

In [ ]:
daily_timeline = df.groupby('only_date').count()['messages'].reset_index()
daily_timeline


In [ ]:
plt.figure(figsize=(10,10))
plt.xticks(rotation="vertical")
plt.plot(daily_timeline['only_date'],daily_timeline['messages'])

In [ ]:
df.head()

In [ ]:
df['day_name'] = df['dates'].dt.day_name()
df

In [ ]:
busy = df['day_name'].value_counts()
busy.index

In [ ]:
sns.barplot(x=busy.index,y=busy.values)

In [ ]:
busy_month = df['month'].value_counts()
busy_month

In [ ]:
plt.xticks(rotation="vertical")
sns.lineplot(x = busy_month.index,y = busy_month.values)


In [ ]:
df[['day_name','hour']]['hour']

In [ ]:
timings = []
for hour in df[['day_name','hour']]['hour']:
    if hour == 23:
        timings.append(str(hour) + "-" + str('00'))
    elif hour == 0:
        timings.append(str('00')+"-"+str(hour+1))
    else:
        timings.append(str(hour)+"-"+str(hour+1))
timings

In [ ]:
df['time_periods'] = timings 
df.head()

In [ ]:
df['time_periods'].value_counts()

In [ ]:
weekely_activity = df.pivot_table(index='day_name',columns="time_periods",values='messages',aggfunc = 'count').fillna(0)

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(weekely_activity,cmap ='seismic')